<a href="https://colab.research.google.com/github/HodaMemar/Patient-Similarity-through-Representation/blob/main/Doc2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import nltk
import time
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk import word_tokenize 
import pandas as pd

In this part, an enriched string is produced for each patient. And to produce the representation, it is given as an input to Algorithm Doc2vec.

In [ ]:

docS_U_TT_R=pd.read_csv('D:\\Embedding\\Code\\A2021\\Stages\\'+title+'.csv')
docS_U_TT_R = docS_U_TT_R.drop('Unnamed: 0', 1)
docS_U_TT_R

In [ ]:
sen = pd.DataFrame({'ID':[ None], 'doc':[None]})
docS_U_TT_R.columns = ['ID', 'doc']
sen=docS_U_TT_R
len(sen)

In [ ]:
import re
ls_ID=sen.ID.unique().tolist()
ls_sen=[]
for i in range(len(ls_ID)):
    s=str(sen.loc[sen['ID']==ls_ID[i],['doc']].values[0])

    ls_sen.append(s)
sen['cleanDoc']=pd.DataFrame(ls_sen)

In [ ]:
from string import digits 
  
def remove(list): 
    remove_digits = str.maketrans('', '', digits) 
    list = [i.translate(remove_digits) for i in list] 
    return list

In [ ]:
start = time.time()
data =list(sen.cleanDoc)

tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]

end = time.time()
print(end - start)

In this section, model doc2vec is used to learn the representation of input information, for more information please refer to :
https://www.tutorialspoint.com/gensim/gensim_doc2vec_model.htm

In [ ]:
start = time.time()
max_epochs = 10
vector_size = 200
alpha = 0.025

model = Doc2Vec(vector_size=200,
                alpha=alpha, 
                min_alpha=0.0001,
                min_count=5,
                dm =0, # This parameter controls the behavior of the algorithm.
                negative = 5,
                #hs=1,
                workers=20,
                dm_concat = 1,
                sample =1e-5)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    try:
        print('iteration {0}'.format(epoch))
        model.train(tagged_data,
                    total_examples=model.corpus_count,
                    epochs=model.iter)
        # decrease the learning rate
        model.alpha -= 0.0002
        # fix the learning rate, no decay
        model.min_alpha = model.alpha
    except:
        print('1')

model.save("d2v.model")
end = time.time()
print("Model Saved in " + str(end - start))

In this part, the output resulting from the implementation of running Doc2vec, which is a vector with 200 dimentions, is connected with the admission number (admission_Id) of the same patient

In [ ]:
list_Subject_id=sen.ID.unique().tolist()
list_total=[]
list_columns_name=[]
for i in range(len(list_Subject_id)):
    #list_vec.append([list_sen[i][0],list_sen[i][1],model.docvecs[i],'Malignant',1])
    list_vec=[]
    for j in range(vector_size):
        list_vec.append(model.docvecs[i][j])   
    list_total.append(list_vec)    


for i in range(vector_size):
    list_columns_name.append('f'+str(i))
    
    
EmbeddedVecor=pd.DataFrame(list_total,columns=list_columns_name)
EmbeddedVecor['admission_Id']=pd.DataFrame(list_Subject_id)